# Metric-based Quantum Architecture Search

## Required imports

In [62]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [51]:
!pip install torch torchquantum

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install matplotlib

  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached fonttools-4.37.2-py3-none-any.whl (959 kB)
  Using cached kiwisolver-1.4.4-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


## Random circuits

In [ ]:
x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

num_params = [12, 16, 20, 24]

num_qubits = 4
num_embeds = 0

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random_fe/angle_1/fmnist_4/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(25):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        circ_gates = ['ry'] * 4 + ['rx'] * 4 + ['rz'] * 4 + ['rx'] * 4 + circ_gates
        gate_params = [[0], [1], [2], [3]] * 4 + gate_params
        inputs_bounds = [j for j in range(16)] + [16 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for i in range(16)] + weights_bounds
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0, 1], 'exp', 'adjoint') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 16000, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list)   

Step 1 | Sliding Loss Window : 0.8124678332931821
Step 51 | Sliding Loss Window : 1.540250109713958
Step 101 | Sliding Loss Window : 1.4744395729477893
Step 151 | Sliding Loss Window : 1.5522063204011483
Step 201 | Sliding Loss Window : 1.3709939625803134
Step 251 | Sliding Loss Window : 1.2944618541405672
Step 301 | Sliding Loss Window : 1.4842890619247353
Step 351 | Sliding Loss Window : 1.286597068073609
Step 401 | Sliding Loss Window : 1.3875165418928161
Step 451 | Sliding Loss Window : 1.3016864657420202
Step 501 | Sliding Loss Window : 1.5044945513477375
Step 551 | Sliding Loss Window : 1.425257468701438
Step 601 | Sliding Loss Window : 1.2380190184607012
Step 651 | Sliding Loss Window : 1.4237690430221313
Step 701 | Sliding Loss Window : 1.2252290338299516
Step 751 | Sliding Loss Window : 1.4193447714747185
Step 801 | Sliding Loss Window : 1.3835780172858108
Step 851 | Sliding Loss Window : 1.4090557696801813
Step 901 | Sliding Loss Window : 1.3456414117808353
Step 951 | Sliding

## Train humand esigned circuits

In [ ]:
from datasets import load_dataset
from create_human_design_circs import generate_human_design_circ
from train_circ import mse_loss, mse_vec_loss

import os
import numpy as np

dataset = 'fmnist_2'
num_reps = 1

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

num_params = [16, 20, 24, 28, 32]
num_embed_layers = 1

num_train_steps = 8000
batch_size = 1
loss_fn = mse_loss

ours_dir = './human_design/amp_basic/fmnist_2/'

if not os.path.exists(ours_dir):
    os.mkdir(ours_dir)

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train.reshape((len(x_train), 1, len(x_train[0])))
x_test = x_test.reshape((len(x_test), 1, len(x_test[0])))
# x_train = x_train.reshape((len(x_train), num_embed_layers, num_qubits))
# x_test = x_test.reshape((len(x_test), num_embed_layers, num_qubits))

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RY], [qml.RX]] * 10
enc_layer_options = [[None, True], [None, True]] * 2
# enc_layer_options = [['Y'], ['X']] * 2

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // num_qubits
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'amp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options[:num_embed_layers],
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, num_train_steps, 0.05, batch_size, loss_fn, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([loss_fn(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(np.mean(val_loss))
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 0.11845170016908314
Step 51 | Sliding Loss Window : 0.7066787155511105
Step 101 | Sliding Loss Window : 0.4954576217530123
Step 151 | Sliding Loss Window : 0.39850094414450543
Step 201 | Sliding Loss Window : 0.3995942785084782
Step 251 | Sliding Loss Window : 0.3039405325413277
Step 301 | Sliding Loss Window : 0.26855777588001756
Step 351 | Sliding Loss Window : 0.23778640169019563
Step 401 | Sliding Loss Window : 0.28814245501643276
Step 451 | Sliding Loss Window : 0.2613440891739329
Step 501 | Sliding Loss Window : 0.23744973174819306
Step 551 | Sliding Loss Window : 0.31265500410766683
Step 601 | Sliding Loss Window : 0.25173919826808255
Step 651 | Sliding Loss Window : 0.2516009423346549
Step 701 | Sliding Loss Window : 0.25389721509638613
Step 751 | Sliding Loss Window : 0.24949941351071778
Step 801 | Sliding Loss Window : 0.22437476837397705
Step 851 | Sliding Loss Window : 0.2869253820530329
Step 901 | Sliding Loss Window : 0.24812377924642742
Ste

In [ ]:
from datasets_nt import load_dataset
from create_human_design_circs import generate_human_design_circ

dataset = 'fmnist_2'
num_reps = 1

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]

ours_dir = './human_design/iqp_basic/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train.reshape((len(x_train), 2, -1))
x_test = x_test.reshape((len(x_test), 2, -1))

num_params = [16, 20, 24, 28, 32]
params_per_layer = 4

num_embed_layers = 4

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RX], [qml.RY]] * 4
enc_layer_options = [[1, None], [1, None]]

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // params_per_layer
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'iqp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options,
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, 5490, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(val_loss < 1)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

[2022-09-18 03:21:44.611 tensorflow-2-3-cpu-py3-ml-t3-large-ec8b7fc7e5cf4fdb2037b69e2c1f:23 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-09-18 03:21:44.673 tensorflow-2-3-cpu-py3-ml-t3-large-ec8b7fc7e5cf4fdb2037b69e2c1f:23 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Step 1 | Sliding Loss Window : 1.4672621210427153
Step 51 | Sliding Loss Window : 1.1051057725183246
Step 101 | Sliding Loss Window : 1.0788874347672104
Step 151 | Sliding Loss Window : 1.1855625082665529
Step 201 | Sliding Loss Window : 1.0923175041488502
Step 251 | Sliding Loss Window : 1.0509031722930369
Step 301 | Sliding Loss Window : 1.0644686067980433
Step 351 | Sliding Loss Window : 1.1391947584364164
Step 401 | Sliding Loss Window : 0.957039160418616
Step 451 | Sliding Loss Window : 1.0111143232566306
Step 501 | Sliding Loss Window : 1.1359713614329854
Step 551 | Sliding Loss Window : 0.9465878349872575
Step 601 | Slidin

In [ ]:
!python supernet/train_search_fmnist.py --warmup_epochs 0 --steps 50000 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 3 --n_encode_layers 4 --save "12_params" --data "./experiment_data/fmnist_4/" --save_dir "./supernet/fmnist_4"

2022-09-23 05:08:44.378853: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-09-23 05:08:44.379001: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-09-23 05:08:44.410566: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(8000, 4, 4) (2000, 4, 4)
subnet: [90641, 135497, 186494], expert_idx: 0
2022-09-23 05:08:57.973045: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 05:08:57.980726: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2499990000 H

## supernet

In [9]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [5]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_batched_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'moons_300'
main_dir = './supernet/moons/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 2)

# inds = np.random.choice(len(x_test), 400, False)
# x_test = x_test[inds]
# y_test = y_test[inds]

num_qubits = 2
num_cnot_configs = 4
num_circs = 2500
num_embed_layers = 2

# device_name = 'ibm_nairobi'
dev = qml.device('lightning.qubit', wires=num_qubits)
# dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [16]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0, 1], 'exp')
        
        val_exp_list = []
        
        for j in range(np.ceil(len(x_test) / 4).astype('int32')):
            curr_batch_params = np.concatenate([[curr_params] for k in range(min(4, len(x_test) - 4 * j))], 0)
            
            val_exp_list.append(circ(x_test[4 * j:min(len(x_test), 4 * j + 4)], curr_batch_params))
            
        val_exps = np.concatenate(val_exp_list)
        val_losses = np.sum(np.power(val_exps - y_test, 2), 1)
        val_loss = np.mean(val_losses)
        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0) == 2)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

ibmqfactory.load_account:WARNING:2022-10-03 03:30:38,162: Credentials are already in use. The existing account in the session will be replaced.


KeyboardInterrupt: 

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

dataset = 'moons_300'
main_dir = './supernet/moons/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 2)

num_qubits = 2
num_cnot_configs = 4
num_circs = 2500
num_embed_layers = 2
meas_qubits = [0]
num_steps = 2400

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [2, 4, 6, 8, 10, 12, 14, 16]
search_nums = [100]
num_trials = [25]

device_name = 'ibmq_lima'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
    circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
    circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/{}/search_{}'.format(device_name, search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
            np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

ibmqfactory.load_account:WARNING:2022-10-03 03:53:51,877: Credentials are already in use. The existing account in the session will be replaced.


0.7583333333333333 0.8095350583394368 [30]
Step 1 | Sliding Loss Window : 0.2625957513150957
Step 51 | Sliding Loss Window : 0.8940166125992647
Step 101 | Sliding Loss Window : 0.8207359836993764
Step 151 | Sliding Loss Window : 0.8874163426869462
Step 201 | Sliding Loss Window : 0.8829311148020272
Step 251 | Sliding Loss Window : 1.2068792765359282
Step 301 | Sliding Loss Window : 0.9763655813490786
Step 351 | Sliding Loss Window : 0.9152893878178131
Step 401 | Sliding Loss Window : 0.9487559791972767
Step 451 | Sliding Loss Window : 0.794616805583784
Step 501 | Sliding Loss Window : 0.8734648782680706
Step 551 | Sliding Loss Window : 0.829056698750842
Step 601 | Sliding Loss Window : 0.8414604038578966
Step 651 | Sliding Loss Window : 0.8484074528905264
Step 701 | Sliding Loss Window : 1.1128946987115542
Step 751 | Sliding Loss Window : 1.0781747821055017
Step 801 | Sliding Loss Window : 0.8531716587872745
Step 851 | Sliding Loss Window : 1.0734172648229288
Step 901 | Sliding Loss Wi

# uor metric

In [32]:
from datasets_nt import load_dataset
from create_gate_circs import create_batched_gate_circ, generate_true_random_gate_circ
from metrics import compute_reduced_similarity

search_params = [4, 8, 12]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 4
num_embeds = 8
meas_qubits = [0]

x_train, y_train, x_test, y_test = load_dataset('bank', 'angle', 2)
main_dir = './ours/bank/'

class_0_sel = np.random.choice(549, 16, False)
class_1_sel = np.random.choice(549, 16, False) + 549
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

np.savetxt(main_dir + '/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours/bank/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []
    
    for i in range(2500):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
    
        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((32, 32))
        circ_t_mean_mat = np.zeros((32, 32))

        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(32)]).reshape((32, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > np.sort(mat.flatten())[511]
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / 32
            circ_t_mean_mat += t_mat / 32
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Our metric + fixed embedding

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_batched_gate_circ, generate_true_random_gate_circ
from metrics import compute_reduced_similarity

search_params = [12]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 2
num_embeds = 0
meas_qubits = [0]

x_train, y_train, x_test, y_test = load_dataset('moons_300', 'angle', 2)
main_dir = './ours_fe/moons/angle_2'

class_0_sel = np.random.choice(240, 16, False)
class_1_sel = np.random.choice(240, 16, False) + 240
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]

np.savetxt(main_dir + '/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = main_dir + '/{}_params_graph2'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(75000):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        circ_gates = ['ry'] * 2 + ['rx'] * 2 + circ_gates
        gate_params = [[0], [1]] * 2 + gate_params
        inputs_bounds = [j for j in range(4)] + [4 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for i in range(4)] + weights_bounds
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((32, 32))
        circ_t_mean_mat = np.zeros((32, 32))

        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(32)]).reshape((32, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - 32) / 996)
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / 32
            circ_t_mean_mat += t_mat / 32
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

/usr/local/lib/python3.7/site-packages/numpy/lib/npyio.py:1378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


## train circuits search using our method

In [26]:
from datasets_nt import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss

search_nums = [100]
param_nums = [4, 8, 12, 16, 20]

dataset = 'bank'
num_reps = 2

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]
mat_size = 1024

device_name = 'ibmq_lima'

ours_dir = './ours/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        noise_scores = []
        
        for i in range(2500):
            noise_scores.append(np.genfromtxt(ours_dir + '/{}_params/circ_{}/noise_metric/{}/metric_tvd_score.txt'.format(param_num, i + 1, device_name)))
            
        noise_scores = np.array(noise_scores)
        
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))
        perf_scores = (mat_size - mean_t_mat_scores) / mat_size
        combined_scores = np.multiply(perf_scores, noise_scores)
        
        circ_inds = np.random.permutation(2500)
        
        curr_dir = ours_dir + '/{}_params/search_{}_{}'.format(param_num, search_num, device_name)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = circ_inds[range(j * 100, j * 100 + 100)]
            sel_ind = sel_inds[np.argmax(combined_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', combined_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])
            np.savetxt(curr_trial_dir + '/sel_circuit_score.txt', [combined_scores[sel_ind]])
            
            print(sel_inds, combined_scores[sel_inds])
            print(sel_ind, combined_scores[sel_ind])
            print(noise_scores[sel_ind], perf_scores[sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
        
            losses_list = []
            accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 2400, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
#                 acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                losses_list.append(val_loss)
                accs_list.append(acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          

ValueError: operands could not be broadcast together with shapes (77500,) (2500,) 

In [28]:
perf_scores.shape

(77500,)

## compute metrics for correlation circuits

In [ ]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets import load_dataset

ideal = np.concatenate((np.ones((32, 32)), np.zeros((32, 32)), np.zeros((32, 32)), np.zeros((32, 32))))
ideal_2 = np.concatenate((np.zeros((32, 32)), np.ones((32, 32)), np.zeros((32, 32)), np.zeros((32, 32))))
ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

curr_dir = './experiment_data/fmnist_4/trained_circuits'
num_qubits = 4
meas_qubits = [0, 1]

x_train, y_train, x_test, y_test = load_dataset('fmnist_4', 'angle', 1)

class_0_sel = np.random.choice(2000, 32, False)
class_1_sel = np.random.choice(2000, 32, False) + 2000
class_2_sel = np.random.choice(2000, 32, False) + 4000
class_3_sel = np.random.choice(2000, 32, False) + 6000
sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

sel_data = x_train[sel_inds]

d2_min_scores = []
d2_mean_scores = []
d2_var_scores = []
    
d2_t_min_scores = []
d2_t_mean_scores = []
d2_t_var_scores = []

mean_mat_scores = []
mean_t_mat_scores = []

np.savetxt(curr_dir + '/fid_mats_sel_data.txt', sel_data)

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
    
    batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                        weights_bounds, meas_qubits, 'matrix') 
    
    params = 2 * np.pi * np.random.sample((32, weights_bounds[-1]))
    
    if not os.path.exists(circ_dir + '/fid_mats'):
        os.mkdir(circ_dir + '/fid_mats')
    
    np.savetxt(circ_dir + '/fid_mats/metric_params.txt', params) 
    
    circ_d2_scores = []
    circ_d2_t_scores = []
    circ_mean_mat = np.zeros((128, 128))
    circ_t_mean_mat = np.zeros((128, 128))
    
    for j in range(32):
        curr_params = np.concatenate([params[j] for k in range(128)]).reshape((128, weights_bounds[-1]))
        mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
        
        t_mat = mat > ((np.sum(mat) - 128) / 16256)
        
        diff_mat = mat - ideal
        diff_2d = np.sum(np.multiply(diff_mat, diff_mat))
            
        diff_t_mat = t_mat - ideal
        diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))
            
        circ_d2_scores.append(diff_2d)
        circ_d2_t_scores.append(diff_2d_t)
            
        circ_mean_mat += mat / 32
        circ_t_mean_mat += t_mat / 32

    np.savetxt(circ_dir + '/fid_mats/d2_scores_128.txt', circ_d2_scores)
    np.savetxt(circ_dir + '/fid_mats/d2_t_scores_128.txt', circ_d2_t_scores)
        
    d2_min_scores.append(np.min(circ_d2_scores))
    d2_mean_scores.append(np.mean(circ_d2_scores))
    d2_var_scores.append(np.var(circ_d2_scores))
        
    d2_t_min_scores.append(np.min(circ_d2_t_scores))
    d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
    d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    diff_mean_mat = ideal - circ_mean_mat
    diff_t_mean_mat = ideal - circ_t_mean_mat
        
    mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
    mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))        
    
    print(i)
        
np.savetxt(curr_dir + '/d2_mean_scores_128.txt', d2_mean_scores)
np.savetxt(curr_dir + '/d2_min_scores_128.txt', d2_min_scores)
np.savetxt(curr_dir + '/d2_var_scores_128.txt', d2_var_scores)
    
np.savetxt(curr_dir + '/d2_t_mean_scores_128.txt', d2_t_mean_scores)
np.savetxt(curr_dir + '/d2_t_min_scores_128.txt', d2_t_min_scores)
np.savetxt(curr_dir + '/d2_t_var_scores_128.txt', d2_t_var_scores)
    
np.savetxt(curr_dir + '/d2_mean_mat_scores_128.txt', mean_mat_scores)
np.savetxt(curr_dir + '/d2_mean_t_mat_scores_128.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9


## noise metric for corlrleaiton circuits

In [66]:
from importlib import reload

import metrics

reload(metrics)

<module 'metrics' from '/root/metrics.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_2'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_quito'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-11-05 17:07:01,531: Credentials are already in use. The existing account in the session will be replaced.
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (69296.59853223679 > 2 * 17329.356686709158). Truncating to maximum value.
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (69296.59853223679 > 2 * 17329.356686709158). Truncating to maximum value.
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (69296.59853223679 > 2 * 17329.356686709158). Truncating to maximum value.
/usr/local/lib/python3.7/site-packages/qiskit_

0.808197021484375 0.8283920288085938 0
0.6967315673828125 0.72320556640625 1
0.8394775390625 0.85003662109375 2
0.7586441040039062 0.7827606201171875 3
0.748504638671875 0.7749176025390625 4
0.797149658203125 0.8062591552734375 5
0.8660125732421875 0.8785171508789062 6
0.8473739624023438 0.8447952270507812 7
0.7137908935546875 0.7598800659179688 8
0.8480758666992188 0.8467941284179688 9
0.5645599365234375 0.6921157836914062 10
0.7797164916992188 0.801300048828125 11
0.8901214599609375 0.9065704345703125 12
0.8502960205078125 0.8595733642578125 13
0.8442764282226562 0.8451156616210938 14
0.785552978515625 0.7784576416015625 15
0.7772064208984375 0.8285598754882812 16
0.77203369140625 0.7636032104492188 17
0.8231658935546875 0.8378372192382812 18
0.7422714233398438 0.7538833618164062 19
0.8855972290039062 0.8899688720703125 20
0.8676071166992188 0.8918228149414062 21
0.8130645751953125 0.8391799926757812 22
0.7215576171875 0.7065353393554688 23
0.8202362060546875 0.8526077270507812 24
0.

## train correlation crcuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_oslo'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(0, 200):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

0.5700597719844201 0.6162277030944824 1


In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

print(x_train.shape)

num_qubits = 4
num_embeds = 16
num_params = 32

for i in range(95, 110):
    curr_dir = './experiment_data/mnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 16000, 0.05, 1, mse_vec_loss, verbosity=127300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

(8000, 16)
Step 1 | Sliding Loss Window : 1.2154147543935532
Step 51 | Sliding Loss Window : 1.4029157730350812
Step 101 | Sliding Loss Window : 1.2635895609147427
Step 151 | Sliding Loss Window : 1.2228109763348884
Step 201 | Sliding Loss Window : 1.1926821664233804
Step 251 | Sliding Loss Window : 1.1362221280606633
Step 301 | Sliding Loss Window : 1.174353337056863
Step 351 | Sliding Loss Window : 1.1754448807464177
Step 401 | Sliding Loss Window : 1.1033157613440856
Step 451 | Sliding Loss Window : 1.2044207562622564
Step 501 | Sliding Loss Window : 1.1886213261759782
Step 551 | Sliding Loss Window : 1.1077056219968855
Step 601 | Sliding Loss Window : 1.1793181655662253
Step 651 | Sliding Loss Window : 1.110171771895772
Step 701 | Sliding Loss Window : 1.1918039718108282
Step 751 | Sliding Loss Window : 1.1877711864453313
Step 801 | Sliding Loss Window : 1.2005887694686992
Step 851 | Sliding Loss Window : 1.0875460821717222
Step 901 | Sliding Loss Window : 1.1853579034144615
Step 9

In [21]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=100, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(100, 133):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 0.072265625
Step 1001 | Accuracy: 0.16015625
Step 2001 | Accuracy: 0.212890625
Step 3001 | Accuracy: 0.255859375
Step 4001 | Accuracy: 0.2080078125
Step 5001 | Accuracy: 0.251953125
Step 6001 | Accuracy: 0.251953125
Step 7001 | Accuracy: 0.2744140625
Step 8001 | Accuracy: 0.2783203125
Step 9001 | Accuracy: 0.271484375
Step 10001 | Accuracy: 0.2626953125
Step 11001 | Accuracy: 0.240234375
Step 12001 | Accuracy: 0.2685546875
Step 13001 | Accuracy: 0.2724609375
Step 14001 | Accuracy: 0.2744140625
Step 15001 | Accuracy: 0.2431640625
Step 16001 | Accuracy: 0.25
Step 17001 | Accuracy: 0.2783203125
Step 18001 | Accuracy: 0.26171875
Step 19001 | Accuracy: 0.2412109375
Step 20001 | Accuracy: 0.294921875
Step 21001 | Accuracy: 0.2666015625
Step 22001 | Accuracy: 0.2705078125
Step 23001 | Accuracy: 0.2802734375
Step 24001 | Accuracy: 0.2890625
Step 25001 | Accuracy: 0.30078125
Step 26001 | Accuracy: 0.3134765625
Step 27001 | Accuracy: 0.2900390625
Step 28001 | Accuracy: 0.28515